In [3]:
import pandas as pd
import re
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord


In [93]:
clusters_names_csv = open('clustersname.csv','w')

for dirpath, dirnames, files in os.walk('./'):
    for file_name in files:
        file_path = os.path.join(dirpath, file_name)
        if dirpath == './knownclusterblast':
            str_fn = str(file_name).replace('.txt','').replace('c','').split('_')[-1]            
            file = open(file_path,'r')            
            lines = file.readlines()
            cluster_name = ''
            for index, lin in enumerate(lines):
                if lin.startswith('Significant hits'):
                    if index > 0:
                        previous = lines[index - 1]
                    if index < (len(lines) - 1):
                        next_ = lines[index + 1]
                        if next_.startswith('1.'):
                            cluster_name = next_.replace('\t','_').split('_')[-1]
                        else:
                            cluster_name = 'region'
            clusters_names_csv.write(str_fn+',')
            if cluster_name != None:
                clusters_names_csv.write(str(cluster_name)+'\n')
        else:
            pass
clusters_names_csv.close()

In [93]:
clust_name = pd.read_csv('clustersname.csv', 
                         skip_blank_lines=True, 
                         header=None)

clust_name.rename(columns={0:'clust_num',1:'nome'}, inplace = True)
#print(clust_name.info)

df2 = clust_name.sort_values(by='clust_num')
df2.fillna('region', inplace = True)
df2.reset_index(drop=True, inplace=True)
df2.index = df2.index + 1
print(df2)
        
clusters_names_csv.close()

    clust_num                       nome
1           1                     region
2           2                  surfactin
3           3  butirosin A / butirosin B
4           4                     region
5           5              macrolactin H
6           6                 bacillaene
7           7                   fengycin
8           8                     region
9           9                     region
10         10                 difficidin
11         11                     region
12         12                  bacilysin


In [120]:
for dirpath, dirnames, files in os.walk('./'):
    for dirname in dirnames:
        pastas = dirname
    for file_name in files:
        
        if file_name.startswith('NZ_CP059318.1.region0') and file_name.endswith('.gbk'):
            
            cds_motifs_list = []
            cds_motifs_counts = {}
            
            reg_num0 = file_name.split('.')[-2][-2:]
            
            
            acc_clust = file_name.split('.')[0]
                      
            
            #print(cluster_name)
            #print(int(reg_num0))
            
            record = SeqIO.read(file_name, "genbank")
            print('\n'+file_name)
            for features in record.features:
                if features.type == 'CDS_motif':
                    if features.qualifiers.get('note') != None:
                        cdsmotif_note = features.qualifiers.get('note')[0]
                        cdsmotif_note_f = cdsmotif_note.split(':')[1].strip()
                        if cdsmotif_note_f not in cds_motifs_list:
                            cds_motifs_list.append(cdsmotif_note_f)
                            cds_motifs_counts[cdsmotif_note_f] = 0
                        cds_motifs_counts[cdsmotif_note_f] += 1
                        
                    elif features.qualifiers.get('aSTool') != None:
                        #print(features.qualifiers.get('aSTool'))
                        labels_motifs = str(features.qualifiers.get('label')).strip('[]"\'"')
                        if labels_motifs not in cds_motifs_list:
                            cds_motifs_list.append(labels_motifs)
                            #print(labels_motifs)
                            cds_motifs_counts[labels_motifs] = 0
                        cds_motifs_counts[labels_motifs] += 1                        
                        
                    else:
                        cdsmotif_note = None 
                        cdsmotif_note_f = None
                        
                    CDS_motif = open('features_matrices/CDS_motifs.csv', 'w')                 
                     
            #print(cds_motifs_counts)
            #print(len(cds_motifs_list))
            
            df1 = pd.DataFrame([cds_motifs_counts], index=[1])
            
            print(reg_num0)
            print(cluster_name)
            
            cluster_name_ = df2.loc[df2["clust_num"] == int(reg_num0), "nome"]
            
            print(cluster_name_)
            
            df1['nome'] = cluster_name_
            
            print(df1)
            print('\n')
            
            
            CDS_motif.close()                
                #if features.type == 'CDS':
                #    if features.qualifiers.get('gene_functions') != None:
                        #print(features.qualifiers.get('gene_functions'))
                #    else:
                        #print(features.qualifiers.get('product'))


NZ_CP059318.1.region007.gbk
07
6    bacillaene
Name: nome, dtype: object
7    fengycin
Name: nome, dtype: object
   PKSI-KR_m1  NRPS-te1  NRPS-A_a10  NRPS-A_a8  NRPS-A_a6  NRPS-A_a3  \
1           2         2           9         18         18         20   

   NRPS-A_a2  C67_DCL_14fromHMM  C5_DCL_263-294  C4_DCL_171-183  ...  \
1         17                  7               7               4  ...   

   PKSI-KS_m3  PKSI-AT-M_m3  PKSI-AT-mM_m1  C1_DCL_004-017  \
1           1             1              1               4   

   C2_starter_024-063  C1_starter_004-017  C67_starter_14fromHMM  \
1                   2                   2                      1   

   C5_starter_265-294  C3_starter_134-155  nome  
1                   1                   1   NaN  

[1 rows x 36 columns]



NZ_CP059318.1.region011.gbk
11
6    bacillaene
Name: nome, dtype: object
11    region
Name: nome, dtype: object
   PKSI-KR_m1  C3_dual_135-156  NRPS-te1  NRPS-beforete1  NRPS-A_a10  \
1           4           

In [ ]:
def processCDSMotif(feature):
    note_text = feature.qualifiers['note'][0]
    if "(" not in note_text: #CDS name not formatted properly
        return None
    motif_name = note_text[0:note_text.index("(")-1]
    return motif_name

def processCDS(feature):
    smCOG_type = None
    for note in feature.qualifiers["note"]:
        if "smCOG" in note:
            if ":" not in note or "(" not in note:
                continue
            smCOG_type = note[note.index(":")+2:note.index("(")-1]
    return smCOG_type

def processPFAM(feature, score_cutoff=20):
    score = float(feature.qualifiers["score"][0])
    if score <score_cutoff:
        return None, None
    domain_description = feature.qualifiers["description"][0]
    pfam_id = feature.qualifiers["db_xref"][0]
    pfam_id = pfam_id[pfam_id.find(" ")+1:len(pfam_id)]
    return domain_description, pfam_id

In [ ]:
for i in cds_motifs_counts:
                for j in cds_motifs_list:
                    if j in cds_motifs_counts:
                        CDS_motif.write(str(cds_motifs_counts[i]) + ",")
                    else:
                        CDS_motif.write("0,")
                CDS_motif.write("\n")        
            print(cds_motifs_counts)
            print(len(cds_motifs_list))
        CDS_motif.close()  